**Importing Library and Installing Packages**

In [14]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 77 kB 3.2 MB/s 


In [15]:
import os
import zipfile
import matplotlib.pyplot as plt
import tensorflow as tf
import pathlib
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**Download Dataset**

In [16]:
!kaggle datasets download techsash/waste-classification-data

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


**Saving images**

In [17]:
path = os.getcwd()

train_dir = path + '/TRAIN'
validation_dir = path + '/TEST'

train_organic_dir = os.path.join(train_dir, 'O')
train_anorganic_dir = os.path.join(train_dir, 'R')

validation_organic_dir = os.path.join(validation_dir, 'O')
validation_anorganic_dir = os.path.join(validation_dir, 'R')

**Training and Validation Generators**

In [18]:
def train_val_generators(train_dir, validation_dir):

  train_datagen = ImageDataGenerator(rescale=1/255)
  train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                      batch_size=64,
                                                      class_mode='binary',
                                                      target_size=(150, 150))
  
  validation_datagen = ImageDataGenerator(rescale=1/255)

  validation_generator = validation_datagen.flow_from_directory(directory=validation_dir,
                                                                batch_size=32,
                                                                class_mode='binary',
                                                                target_size=(150, 150))

  return train_generator, validation_generator

train_generator, validation_generator = train_val_generators(train_dir, validation_dir)

Found 22564 images belonging to 2 classes.
Found 2513 images belonging to 2 classes.


**Model** (Result 92%)

In [19]:
model = tf.keras.Sequential([
    # Input_shape = (150, 150, 3)
    tf.keras.layers.Conv2D(32, (3,3), activation='relu' ,input_shape=(150, 150, 3)),
    # Pool_size = (2,2)
    tf.keras.layers.MaxPooling2D(2, 2),
     
    # Flatten the output layer to 1 dimension
    tf.keras.layers.Flatten(),

    # Output arrays of size 64
    tf.keras.layers.Dense(64, activation='relu',kernel_initializer='he_uniform'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(28, activation='relu',kernel_initializer='he_uniform'),
    tf.keras.layers.Dropout(0.2),
      
    # Using activation sigmoid for 2 clases
    tf.keras.layers.Dense(1, activation='sigmoid')])

# Optimizer using default value
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC(name = 'accuracy')])

**Callbacks** (Optional)

In [20]:
class myCallback(tf.keras.callbacks.Callback):
           
        def on_epoch_end(self, epoch, logs={}):
            if (logs.get('accuracy')) > 0.99: 
                print("\nReached 99% accuracy so cancelling training!")    
                self.model.stop_training = True

# Initiate callbacks
callbacks = myCallback()

**Training**

In [ ]:
history = model.fit(train_generator, validation_data=validation_generator, epochs=10, callbacks=callbacks) 

Epoch 1/10
353/353 [==============================] - 371s 1s/step - loss: 0.4722 - accuracy: 0.8592 - val_loss: 0.3396 - val_accuracy: 0.9367
Epoch 2/10
353/353 [==============================] - 334s 946ms/step - loss: 0.4120 - accuracy: 0.8939 - val_loss: 0.4051 - val_accuracy: 0.9249
Epoch 3/10
353/353 [==============================] - 329s 933ms/step - loss: 0.3769 - accuracy: 0.9116 - val_loss: 0.3190 - val_accuracy: 0.9399
Epoch 4/10
353/353 [==============================] - 329s 931ms/step - loss: 0.3154 - accuracy: 0.9385 - val_loss: 0.3582 - val_accuracy: 0.9329
Epoch 5/10
239/353 [===================>..........] - ETA: 1:42 - loss: 0.2386 - accuracy: 0.9646

**CNN Model Baseline**

In [ ]:
print(model.summary())

Convert to **Tflite** and **h5** (Optional)

In [ ]:
export_dir = 'saved_model/1'

tf.saved_model.save(model,export_dir=export_dir)

# Mode of optimization
mode = "Default" 

if mode == 'Storage':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE
elif mode == 'Speed':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
else:
    optimization = tf.lite.Optimize.DEFAULT

converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

# Set the optimzations
converter.optimizations = [optimization]

# Invoke the converter to finally generate the TFLite model
tflite_model = converter.convert()

tflite_model_file = pathlib.Path('./model.tflite')
tflite_model_file.write_bytes(tflite_model)

# Converting to h5 model
tf.keras.models.save_model(model, "./model.h5")